In [ ]:
import pandas as pd
views_data=pd.read_excel("final_views_data.xlsx", drop_index=True)

In [ ]:
import glob
path="./Articlespk/*.xlsx" #articlepk has all month article content keyed by their article id
files = glob.glob(path)
all_articles = pd.DataFrame() #all articles with article id
for f in files:
    df = pd.read_excel(f)
    print(len(df))
    all_articles = all_articles.append(df,ignore_index=True, sort=True)


In [ ]:
data=pd.merge(views_data,all_articles, on=['news_id'], how="left")

In [ ]:
data.drop(columns=["english"]).to_pickle("./lda_model_data/data.pickle")

In [ ]:
data.head()

In [ ]:
len(data)

In [ ]:
data.nunique()

In [ ]:
data=data.drop(columns=["Unnamed: 0",'pv3hr', 'upv3hr', 'pv3day'])

In [ ]:
data.head(50)

In [ ]:
data.sort_values(by="crtd_date", inplace=True)
#data.reset_index(inplace=True)
data=data.drop(columns=["index"])

In [ ]:
data

In [ ]:
df_list=list(data["english"])

In [ ]:
from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import gensim
import pickle

In [ ]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

In [ ]:
data_words = list(sent_to_words(df_list))

In [ ]:
data.english[1]

In [ ]:
# from nltk.corpus import stopwords
# stop_words = stopwords.words('english')
# stop_words.extend(['from', 'subject', 're', 'edu', 'use'])
# def remove_stopwords(texts):
#     return [[word for word in gensim.utils.simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]
# data_words_nostops = remove_stopwords(data_words)

In [ ]:
documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(data_words)]



In [ ]:
len(documents)

In [ ]:
model = Doc2Vec(vector_size=25, min_count=0,workers=2, alpha=0.025, min_alpha=0.025, dm=0, epochs=1)


In [ ]:
model.min_alpha

In [ ]:
model.build_vocab(documents)

In [ ]:
%%time
for epoch in range(10):
    if epoch%2==0:
        print("training {} ".format(epoch))
    model.train(documents, total_examples=model.corpus_count,epochs=model.epochs)
    model.alpha-=0.002
    model.min_alpha= model.alpha#fixing learning rate throughout 1 epoch
    
    

In [ ]:
model.docvecs.most_similar(positive=[model.infer_vector(data_words[2],alpha=0.025, min_alpha=0.001, steps=55)], topn=8000)

In [ ]:
model.save("./lda_model_data/doc2vec_model")

In [ ]:
model.docvecs.most_similar(2, topn=30)

In [ ]:
data.crtd_date[4968]- timedelta(days=30)

In [ ]:
# %%time
# mupvt10=[]#mean unique page views of top 10 similar articles in last  month
# for i in range(4968,len(data)):#4968,len(data)
#     avg=0
#     count=0
#     print (i)
#     for x in model.docvecs.most_similar(positive=[model.infer_vector(data_words[i],alpha=0.025, min_alpha=0.001, steps=50)], topn=80000):
#         if  data.crtd_date[x[0]] < data.crtd_date[i] and data.crtd_date[x[0]] > data.crtd_date[i]- timedelta(days=30):
#             avg+=data.upv3day[x[0]]
#             count+=1
#         if count==10:
#             break
#     print(avg)
#     print(count)
#     mupvt10.append({str(data.news_id[i]):(avg/count)})
    

In [ ]:
%%time
mupvt10=[]#mean unique page views of top 10 similar articles in last  month
for i in range(4968,len(data)):#4968,len(data) as we are skipping first 10days of december
    avg=0
    count=0
    if i%10000==0:
        print (i)
    for x in model.docvecs.most_similar(i, topn=80000):
        if  data.crtd_date[x[0]] < data.crtd_date[i] and data.crtd_date[x[0]] > data.crtd_date[i]- timedelta(days=30):
            avg+=data.upv3day[x[0]]
            count+=1
        if count==10:
            break
    #print(avg)
    #print(count)
    mupvt10.append(avg/count)
    

In [ ]:
mupvt10

In [ ]:
data2=data[4968:].reset_index().drop(columns=["index"])

In [ ]:
data2.head()

In [ ]:
data2["2msaav10"]=pd.Series(mupvt10)

In [ ]:
data2.to_pickle("data2.pickle")